In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import math
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

In [ ]:
dataset_dir = r"F:\Work\SOCAAutomation\Dataset"
Directory = dataset_dir + "/" + r"Images\09072024_1_1"
FileName = "2024_07_09_11_30_174.jpg"
raw_img = cv.imread(Directory + "/" + FileName)
img = cv.cvtColor(raw_img, cv.COLOR_BGR2GRAY)
img = raw_img

In [3]:
def resize(frame, width = 640, height = 640):
    return cv.resize(frame, (width, height), interpolation = cv.INTER_LINEAR)
def InferenceImage(model, frame):
    results = model(resize(frame), conf=0.3, save=True, show_labels=False, retina_masks=True)
    return results

# Visibility Estimation

In [4]:
def edge_detection(img):    
    gradx = cv.Sobel(img, cv.CV_64F, 1, 0, ksize=3)
    grady = cv.Sobel(img, cv.CV_64F, 0, 1, ksize=3)
    return gradx, grady

def blurimage(img, kernelsize):
    return cv.GaussianBlur(img, (kernelsize, kernelsize), 0)

def convertgray(img):    
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return gray

def preprocess(img, kernelsize):
    gray_img = convertgray(img)
    img_blur = blurimage(gray_img, kernelsize)
    gradx, grady = edge_detection(img_blur)
    grad_mag = cv.magnitude(gradx, grady)
    grad_mag_norm = cv.normalize(grad_mag, None, 0, 255, cv.NORM_MINMAX)
    return grad_mag_norm

def count_edges(target_img, kernelsize, multiplier):
    preprocessed_img = preprocess(target_img, kernelsize)
    _, thresh = cv.threshold(preprocessed_img, multiplier * np.median(preprocessed_img), 255, cv.THRESH_BINARY)             
    total_edges = np.count_nonzero(thresh)
    return total_edges

def edge_strength_ratio(ref_edge, target_edge):
    return np.round((target_edge / ref_edge),2)

In [ ]:
gaussian_kernel = 5
multiplier = 5
ref_edge = count_edges(ref_image, kernelsize, multiplier)
curr_edge = count_edges(image, kernelsize, multiplier)
edge_strength = np.round((curr_edge / ref_edge),2)

In [5]:
def calcFrames(start, end, fps, offset=200):
    startFrame = (int(start[0]) * 60 + int(start[1])) * fps
    endFrame = (int(end[0]) * 60 + int(end[1])) * fps 
    return startFrame-offset, endFrame+offset

# Build Features

In [6]:
def MaskStrength(mask, img):
    """
    Calculates Pixel Strength for mask
    1. Apply mask on the original color image
    2. Calculate average of non zero pixels
    """
    mask_3d = np.expand_dims(mask, axis=-1)
    masked_array = (img * mask_3d).flatten()
    return masked_array[masked_array>0].mean().round(2)

def MaskCentroid(mask):
    """
    Calculates Centroid for mask
    1. Extract indices of non zero pixels in a mask
    2. Calculate average of x and y coordinates
    3. Flip the array, since here x and y corresponds to row and column
    """
    mask_indices = np.where(mask)
    mask_centroid = np.round(np.mean(mask_indices, axis=1))
    flip_centroid = list(np.flip(mask_centroid))
    return flip_centroid


# Main

In [47]:
# Configurations
model_file = r"F:\Work\SOCAAutomation\Models\11_04_2025\best.pt"
video_file = r"F:\Work\SOCAAutomation\Dataset\Converted\04_07_2024\AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772.mp4"
data = pd.read_csv(r"F:\Work\SOCAAutomation\Dataset\Book1.csv")
data_grp = data.groupby(['Folder', 'Video File'])[['Start Time', 'End Time']].agg(list)
start_list = data_grp.iloc[0]['Start Time']
end_list = data_grp.iloc[0]['End Time']
fps = 30

In [ ]:
# Load Model
# REINSTALL ULTRALYTICS 8.2.103 #model = YOLO(model_file, task='segment')


# Load video
video = cv.VideoCapture(video_file)

# Loop over frames

count_frame = 0
frame_referenced = False
subseg_det = false
subseg_ind = 0
start_frame, end_frame = calcFrames(start_list[subseg_ind].split(":"), end_list[subseg_ind].split(":"), fps):
while True:
    ret, frame = video.read()
    count_frame += 1 
    if ret:               
        if frame_referenced is False and count_frame > 10 and count_frame < 100:
            # Build reference image
            ref_image = resize(frame)
            ref_edge = count_edges(ref_image, kernelsize, multiplier)
            frame_referenced = True
            print("Reference Frame detected and Reference edge is {}".format(ref_edge))
        if count_frame >= start_frame and count_frame <= end_frame:
            subseg_det = True
            print("Subsegment {} is detected...".format(subseg_ind + 1))
            
        elif subseg_det is True:
            subseg_det = False
            ind += 1            
            start_frame, end_frame = calcFrames(start_list[subseg_ind].split(":"), end_list[subseg_ind].split(":"), fps)
            print("Start Frame: {} and End Frame {} is recalculated...".format(start_frame,  end_frame))
    else:
        if subseg_det is True:
            print("{} frame is missing".format(count_frame))
            
        



# Reference Images Visibility: Once for one experiment
# Load Reference image
# Estimate visibility

# Loop over subsegments of video

# Inference image

# Build Features



# Model Comparison

In [8]:
test_folder = r"F:\Work\SOCAAutomation\Dataset\Images\Inference\3"
# curr_run = "28_08_2024"
# model_file = r"F:\Work\SOCAAutomation\Models\28_08_2024\best_yolov8m_seg.pt"
# curr_run = "08_04_2025"
# model_file = r"F:\Work\SOCAAutomation\Models\08_04_2025\best_yolov8s_seg.pt"
curr_run = "Test_02_07_2025"
model_file = r"F:\Work\SOCAAutomation\Models\11_04_2025\best.pt"
yaml_path = "custom_bytetrack.yaml"
model = YOLO(model_file)#, task='segment')
# model2 = YOLO(model_file)#, task='segment')
# model3 = YOLO(model_file)#, task='segment')


## Single Image Inferencing

In [21]:
folder = r"F:\Work\SOCAAutomation\Dataset\TestImages"
file = "SingleLargeFire.jpg"
file_path = folder + "/" + file
filename = file.split(".")[0]
test_img = cv.imread(file_path)
test_img_resized = resize(test_img)

In [22]:
results = model(test_img_resized, conf=0.2, iou=0.5, save=True, show_labels=False, retina_masks=True)
bbox_img = test_img_resized.copy()
if results[0].masks is not None:
    masks = np.array(results[0].masks.data)
    combined_mask = np.any(masks, axis=0).astype(np.uint8)
    combined_mask_3ch = cv.merge([combined_mask]*3)
    masked_img = cv.bitwise_and(test_img_resized, test_img_resized, mask=combined_mask)
else:
    masked_img = test_img
    
if results[0].boxes is not None:
    boxes = results[0].boxes
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

        # Draw bounding box on image
        cv.rectangle(bbox_img, (x1, y1), (x2, y2), (0, 255, 0), 2) 

        # Add Confidence score on image
        if box.id is not None:
            id_text = str(box.id.item())
        else:
            id_text = "0"
        cv.putText(bbox_img, ", ".join([id_text, str(np.round(box.conf.item(),2))]),
                org=(x1, y1),  # (x, y) position
                fontFace=cv.FONT_HERSHEY_SIMPLEX,
                fontScale=1,
                color=(0, 0, 255),  # Red color in BGR
                thickness=2) 
cv.imwrite(curr_run + "/" + filename + "_box.jpg", bbox_img)
cv.imwrite(curr_run + "/" + filename + "_mask.jpg", masked_img)


0: 640x640 1 Fire, 18194.8ms
Speed: 2386.2ms preprocess, 18194.8ms inference, 1616.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict24


True

In [16]:
results[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1., 1., 1., 1., 1.])
conf: tensor([0.2733, 0.1492, 0.1291, 0.1170, 0.1113, 0.1034])
data: tensor([[2.4488e+02, 1.2598e+02, 2.9826e+02, 1.7456e+02, 2.7328e-01, 1.0000e+00],
        [2.5758e+02, 1.3991e+02, 2.9446e+02, 1.7508e+02, 1.4916e-01, 1.0000e+00],
        [2.6806e+02, 1.4395e+02, 2.9271e+02, 1.7424e+02, 1.2911e-01, 1.0000e+00],
        [2.7628e+02, 1.5826e+02, 2.9011e+02, 1.7228e+02, 1.1701e-01, 1.0000e+00],
        [2.4717e+02, 1.1870e+02, 3.0491e+02, 1.9030e+02, 1.1129e-01, 1.0000e+00],
        [2.5377e+02, 1.2936e+02, 2.9693e+02, 1.8097e+02, 1.0341e-01, 1.0000e+00]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([6, 6])
xywh: tensor([[271.5680, 150.2717,  53.3832,  48.5773],
        [276.0212, 157.4915,  36.8822,  35.1693],
        [280.3885, 159.0959,  24.6519,  30.2923],
        [283.1989, 165.2719,  13.8281,  14.0199],
        [276.0429, 154.4992,  57.7386,  71.6031],
        [275

In [39]:
test_cnt = 0
for file in os.listdir(test_folder):

    file_path = test_folder + "/" + file
    filename = file.split(".")[0]
#     if int(filename) < 400:
#         continue
    test_img = cv.imread(file_path)
    
    test_img_resized = resize(test_img[50:470, 60:500])
    bbox_img = test_img_resized.copy()
#     results = model(test_img_resized, conf=0.3, save=True, show_labels=False, retina_masks=True)
    results = model.track(test_img_resized, conf=0.3, save=True, show_labels=False, retina_masks=True, tracker=yaml_path, persist=True, stream=False)
    if results[0].masks is not None:
        masks = np.array(results[0].masks.data)
        combined_mask = np.any(masks, axis=0).astype(np.uint8)
        combined_mask_3ch = cv.merge([combined_mask]*3)
        masked_img = cv.bitwise_and(test_img_resized, test_img_resized, mask=combined_mask)
    else:
        masked_img = test_img
    
    
    if results[0].boxes is not None:
        boxes = results[0].boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            # Draw bounding box on image
            cv.rectangle(bbox_img, (x1, y1), (x2, y2), (0, 255, 0), 2) 

            # Add Confidence score on image
            if box.id is not None:
                id_text = str(box.id.item())
            else:
                id_text = "0"
            cv.putText(bbox_img, ", ".join([id_text, str(np.round(box.conf.item(),2))]),
                    org=(x1, y1),  # (x, y) position
                    fontFace=cv.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    color=(0, 0, 255),  # Red color in BGR
                    thickness=2) 

    if not os.path.exists(curr_run):
        os.makedirs(curr_run)
    cv.imwrite(curr_run + "/" + file, bbox_img)
    cv.imwrite(curr_run + "/" + filename + "_mask.jpg", masked_img)
    
#     if test_cnt > 2:
#         break
    test_cnt += 1



0: 640x640 (no detections), 4171.8ms
Speed: 6.0ms preprocess, 4171.8ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 (no detections), 4939.4ms
Speed: 6.0ms preprocess, 4939.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4105.7ms
Speed: 7.0ms preprocess, 4105.7ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4019.6ms
Speed: 7.0ms preprocess, 4019.6ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4112.7ms
Speed: 9.0ms preprocess, 4112.7ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 2 Fires, 4863.4ms
Speed: 8.0ms preprocess, 4863.4ms inference, 13.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segme


0: 640x640 1 Fire, 3815.4ms
Speed: 6.0ms preprocess, 3815.4ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3952.6ms
Speed: 7.0ms preprocess, 3952.6ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3927.5ms
Speed: 7.0ms preprocess, 3927.5ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4374.9ms
Speed: 9.0ms preprocess, 4374.9ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3897.5ms
Speed: 7.0ms preprocess, 3897.5ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3849.5ms
Speed: 7.0ms preprocess, 3849.5ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x6

Results saved to runs\segment\track8

0: 640x640 1 Fire, 4383.9ms
Speed: 8.0ms preprocess, 4383.9ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3893.5ms
Speed: 7.0ms preprocess, 3893.5ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 (no detections), 3950.6ms
Speed: 7.0ms preprocess, 3950.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 (no detections), 3951.6ms
Speed: 8.0ms preprocess, 3951.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 (no detections), 3891.5ms
Speed: 8.0ms preprocess, 3891.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 (no detections), 3983.6ms
Speed: 9.0ms preprocess, 3983.6ms inference, 1.0ms postprocess per image

Speed: 6.0ms preprocess, 3976.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3988.6ms
Speed: 6.0ms preprocess, 3988.6ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3875.5ms
Speed: 7.0ms preprocess, 3875.5ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 2 Fires, 4091.7ms
Speed: 6.0ms preprocess, 4091.7ms inference, 14.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3970.6ms
Speed: 6.0ms preprocess, 3970.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 (no detections), 4226.8ms
Speed: 6.0ms preprocess, 4226.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 (no detections),

Results saved to runs\segment\track8

0: 640x640 (no detections), 3874.5ms
Speed: 7.0ms preprocess, 3874.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3908.5ms
Speed: 7.0ms preprocess, 3908.5ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3888.5ms
Speed: 8.0ms preprocess, 3888.5ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4449.0ms
Speed: 8.0ms preprocess, 4449.0ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3910.5ms
Speed: 7.0ms preprocess, 3910.5ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4003.6ms
Speed: 7.0ms preprocess, 4003.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

Results saved to runs\segment\track8

0: 640x640 1 Fire, 4291.9ms
Speed: 7.0ms preprocess, 4291.9ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3969.6ms
Speed: 7.0ms preprocess, 3969.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4003.6ms
Speed: 7.0ms preprocess, 4003.6ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 3978.6ms
Speed: 12.0ms preprocess, 3978.6ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4106.7ms
Speed: 8.0ms preprocess, 4106.7ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\track8

0: 640x640 1 Fire, 4049.6ms
Speed: 8.0ms preprocess, 4049.6ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)
Results s

In [8]:
test_img = cv.imread(test_folder + "/" + "66.jpg")
test_img_resized = resize(test_img)
results = model(test_img_resized, conf=0.3, save=True, show_labels=False, retina_masks=True)


0: 640x640 2 Fire-Smoke-Sodium-Trays, 8037.2ms
Speed: 301.3ms preprocess, 8037.2ms inference, 599.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict17


# Feature Extraction

In [88]:
# Define list of features to be extracted from the segmented image
feature_list = ['exp', 'video', 'segment', 'frame', 'n_masks', 'mask_area', 'mask_centroid', 'mask_conf', 'mask_strength', 
                'bbox_id', 'bbox_area', 'bbox_centroid', 'bbox_dim']

In [75]:
def checkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
def store_results(results, orig_img, out_dir, file):

    if results[0].masks is not None:
        masks = np.array(results[0].masks.data)
        combined_mask = np.any(masks, axis=0).astype(np.uint8)
        combined_mask_3ch = cv.merge([combined_mask]*3)
        masked_img = cv.bitwise_and(orig_img, orig_img, mask=combined_mask)
    else:
        masked_img = orig_img
    bbox_img = orig_img.copy()

    if results[0].boxes is not None:
        boxes = results[0].boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            # Draw bounding box on image
            cv.rectangle(bbox_img, (x1, y1), (x2, y2), (0, 255, 0), 2) 

            # Add Confidence score on image
            cv.putText(bbox_img, str(np.round(box.conf.item(),2)),
                    org=(x1, y1),  # (x, y) position
                    fontFace=cv.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    color=(0, 0, 255),  # Red color in BGR
                    thickness=2) 

    cv.imwrite(out_dir + "/" + file, bbox_img)
    cv.imwrite(out_dir + "/" + file.split(".")[0] + "_mask.jpg", masked_img)

In [ ]:
def extract_features(results,feature_list):
# Check if atleast one mask exist
    frame = 0
    result = results[0]
    features = {key: [] for key in feature_list}
    if result.masks is not None:
#         features['frame'] = frame
        # Calculate number of masks in the segmented image
        features['n_masks'] = len(result.masks)    

        # Bounding Box Parameters
        bboxes = result.boxes

        # Extract confidence score of all the masks
        features['mask_conf'] = list(np.array(bboxes.conf))

        if bboxes.is_track is True:
            features['bbox_id'] = list(np.array(bboxes.id))
        else:
            features['bbox_id'] = [0] * np.array(bboxes.conf).shape[0]

        bbox_data = np.array(bboxes.data)

        for box in bbox_data:
            # Extract Bounding box dimension [Width, Height] for each box
            width = np.round(abs(box[0] - box[2]))
            height = np.round(abs(box[1] - box[3]))            
            features['bbox_dim'].append([width, height])

            # Extract Bounding box Centroid for each box
            xcoord = np.round(np.mean([box[0], box[2]]))
            ycoord = np.round(np.mean([box[1], box[3]]))
            features['bbox_centroid'].append([xcoord, ycoord])

            # Calculate Bounding box Area for each box
            features['bbox_area'].append(width * height)

        # Mask Parameters
        masks = np.array(result.masks.data, dtype='uint8')
        for mask in masks:
            # Mask Area: Count number of non zero pixels in mask
            features['mask_area'].append(np.count_nonzero(mask))

            # Mask Centroid: Calculate mean of non zero pixel x and y coordinates
            features['mask_centroid'].append(MaskCentroid(mask))

            ## Mask Strength: 
            # Apply mask on the original image
            # Measure average pixel strength excluding non zero pixels
            features['mask_strength'].append(MaskStrength(mask, result.orig_img))
    

    return features

In [107]:
data = pd.read_excel(dataset_dir + "/" + "DatasetAnalysisReport.xlsx", sheet_name = 'Video Analysis')
data.head()

,Folder,Video File,File Path,Start Time,End Time,Fire,Sputtering,Out of Focus,Visibility,Misc,Temp,Folders,Image Selection
0,04_07_2024,AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772,2024-07-04_11_51_08_772,02:35:00,02:40:00,Y,NaN,NaN,NaN,NaN,1,1,Completed
1,04_07_2024,AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772,2024-07-04_11_51_08_772,03:00:00,03:14:00,Y,Y,NaN,NaN,NaN,2,2,Completed
2,04_07_2024,AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772,2024-07-04_11_51_08_772,03:30:00,03:45:00,Y,Y,NaN,L,NaN,3,3,Completed
3,04_07_2024,AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772,2024-07-04_11_51_08_772,03:52:00,03:55:00,Y,Y,NaN,NaN,NaN,4,4,Completed
4,04_07_2024,AXIS 213 - 10.1.5.173 2024-07-04_11_51_08_772,2024-07-04_11_51_08_772,04:05:00,04:11:00,Y,NaN,NaN,NaN,NaN,5,5,Completed


In [126]:
root_dir = dataset_dir + "/" + 'Images\Fullset_reduced'
out_dir = "Run1"
checkdir(out_dir)
test_cnt = 0

for i in range(len(data_filtered)):
    row = data_filtered.iloc[i]
    path = row['Folder'] + "/" + row['File Path'] + "/" + str(row['Folders'])
    segment_path = root_dir + "/" + path
    file_cnt = 0
    print("Inferenceing " + path, " is started...")
    out_path = "/".join([out_dir, path])
    os.makedirs(out_path, exist_ok=True)
    files = sorted([int(x.split(".")[0]) for x in os.listdir(segment_path)])
    features = {key: [] for key in feature_list}
    df_features = pd.DataFrame(features)
    for file_name in files:
        file = str(file_name) + ".jpg"
        file_path = segment_path + "/" + file
        img = cv.imread(file_path)    
        img_resized = resize(img)
        bbox_img = img_resized.copy()
        results = model(img_resized, conf=0.3, save=True, show_labels=False, retina_masks=True, verbose=False)
        d = extract_features(results, feature_list)
        d['exp'], d['video'], d['segment'] = row[['Folder', 'File Path', 'Folders']]
        d['frame'] = file_name
        df_features = pd.concat([df_features, pd.DataFrame([d])])     
        store_results(results, img_resized, out_path, file) 
        if test_cnt > 2:
            break
        test_cnt += 1
    
    df_features.to_csv(out_dir + "/" + row['File Path'] + "_" + str(row['Folders']) + ".csv")

Inferenceing 04_07_2024/2024-07-04_11_51_08_772/3  is started...
Results saved to runs\segment\track8
Results saved to runs\segment\track8
Results saved to runs\segment\track8
Results saved to runs\segment\track8


In [92]:
df_features.groupby('frame').agg({'bbox_id': list, 'mask_centroid':list})

,bbox_id,mask_centroid
frame,,
0.0,"[1.0, 2.0]","[[169.0, 296.0], [246.0, 270.0]]"
1.0,"[1.0, 2.0]","[[166.0, 296.0], [244.0, 265.0]]"
2.0,"[1.0, 2.0]","[[158.0, 300.0], [246.0, 267.0]]"
3.0,"[1.0, 2.0]","[[158.0, 297.0], [245.0, 266.0]]"
4.0,"[1.0, 2.0]","[[159.0, 303.0], [242.0, 264.0]]"
...,...,...
145.0,"[23.0, 25.0]","[[186.0, 289.0], [294.0, 192.0]]"
146.0,[23.0],"[[176.0, 294.0]]"
147.0,[23.0],"[[174.0, 299.0]]"


# Merge images to video 

In [245]:
def Images2Video(image_folder, video_file_name="combine_images.mp4"):
    #image_folder = r"F:\Work\SOCAAutomation\Dataset\Images\25_04_2025"
    out = cv.VideoWriter("combine_images.mp4", cv.VideoWriter_fourcc(*"MJPG"), 1, (640, 640))
    for file in os.listdir(image_folder):
        file_path = image_folder + "/" + file
        frame = resize(cv.imread(file_path))
        out.write(frame)
    out.release()

In [ ]:
Images2Video(r"F:\Work\SOCAAutomation\Dataset\Images\25_04_2025", video_file_name="combine_images.mp4")